# Preprocessing - Experiments

- loading the required information from the json files
- preprocessing
- extracting key phrases

## Requirements

- nltk
- python-rake

In [1]:
!pip3 install python-rake

In [2]:
from __future__ import print_function

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
import json
from os import getcwd
from os import listdir
from os.path import isfile, join
from glob import glob
from literature_utils import DataLoader
import RAKE

## Getting the test files

In [3]:
root_path = join(getcwd(), 'CORD-19-research-challenge', 'biorxiv_medrxiv', 'biorxiv_medrxiv', 'pdf_json')
files = glob(join(root_path, '*.json'))

In [4]:
print(files[0])

/home/tobias/Desktop/Advanced Data Models Projekt/CORD-19-research-challenge/biorxiv_medrxiv/biorxiv_medrxiv/pdf_json/8f5c5631cca22541a8cea87fb688b17f1266b859.json


## RAKE Tests

In [5]:
rake = RAKE.Rake(join(getcwd(), 'stopwords_en.txt'))

In [6]:
phrase_blacklist = [
    'granted medrxiv', 
    'cc-by-nc-', 
    'author/funder', 
    'copyright holder', 
    'peer-reviewed', 
    'preprint', 
    'license', 
    'reuse allowed', 
    'rights reserved', 
    'international license', 
    'cc-by-nc', 
    '0 international license', 
    'medrxiv preprint',
    'cc-by-nc 4']

In [10]:
for f in files[10:20]:
    dl = DataLoader(f)
    text = dl.get_full_text()
    title = dl.get_title()
    keyphrases = rake.run(text, minCharacters = 1, maxWords = 3, minFrequency = 5)
    print(title + "\n")
    print(*[phrase for phrase in keyphrases if phrase[1] >= 1.3 and phrase[0] not in phrase_blacklist], sep='\n')
    print('\n\n')

Epidemiological Tools that Predict Partial Herd Immunity to SARS Coronavirus 2

('sars-cov-2', 5.166666666666666)
('herd immunity', 4.762237762237762)
('population', 1.6363636363636365)
('infected', 1.4444444444444444)
('type', 1.3953488372093024)
('spread', 1.3333333333333333)



Article type: Research Article Title: Under-the-radar dengue virus infections in natural populations of Aedes aegypti mosquitoes Running title: Dengue virus maintenance in mosquito vectors

('florida', 2.176470588235294)



Interferon-α2b treatment for COVID-19

('ifn-α2b', 4.987240829346092)
('covid-19 disease', 4.107843137254902)
('cases treated', 3.75)
('ifn', 2.303030303030303)
('treatment', 1.8888888888888888)
('time', 1.8181818181818181)
('il-6', 1.8125)
('treated', 1.5)
('crp', 1.4)



A Scalable Method of Applying Heat and Humidity for Decontamination of N95 Respirators During the COVID-19 Crisis

('cc-by-nc 4', 9.0)
('test group masks', 6.337108013937282)
('decontamination cycles', 3.5294117647058822